In [1]:
from dotenv import load_dotenv
load_dotenv('../.vscode/.env')

True

In [129]:
recipe_name = 'Classic Apple Pie'
ingredients = '''
- quinoa: 1 cup
- water: 2 cups
- black beans (rinsed and drained): 1 can (15 ounces)
- red bell pepper (diced): 1
- fresh cilantro (chopped): 1/4 cup
- lime juice: 1/4 cup
- olive oil: 2 tablespoons
- ground cumin: 1 teaspoon
- salt: 1/2 teaspoon
- black pepper: 1/4 teaspoon
- avocado (diced, optional): 1
- cherry tomatoes (halved, optional): 1/2 cup
'''
directions= '''
1. Cook Quinoa: Rinse the quinoa under cold water. In a medium saucepan, bring 2 cups of water to a boil. Add quinoa, reduce heat to low, cover, and simmer for about 15 minutes, or until quinoa is tender and water is absorbed. Remove from heat and let it cool.
2. Combine Ingredients: In a large bowl, combine the cooled quinoa, black beans, red bell pepper, and cilantro.
3. Make Dressing: In a small bowl, whisk together lime juice, olive oil, cumin, salt, and black pepper.
4. Mix Salad: Pour the dressing over the quinoa mixture and stir until well combined. If using, gently fold in the avocado and cherry tomatoes.
5. Chill and Serve: Refrigerate the salad for at least 30 minutes before serving. This allows the flavors to meld together.
'''

nutrients='''Total Carbohydrate: 50g, Dietary Fiber: 15g, Sodium: 300mg, Saturated Fat: 2g, Total Fat: 17g, Protein: 12g, Added Sugars: 0g, Total Sugars: 5g'''
total_calorie_estimation='''
- Total Calories for entire recipe : Approximately 1561 calories for the entire recipe. 
- Calories for each ingredient:
    Quinoa: 712 calories for 1 cup cooked (from 185g of dry quinoa)
    Black Beans: 227 calories per can
    Red Bell Pepper: 37 calories
    Cilantro: Negligible
    Lime Juice: 8 calories per 1/4 cup
    Olive Oil: 239 calories per 2 tablespoons
    Cumin, Salt, Black Pepper: Negligible
    Avocado: 322 calories
    Cherry Tomatoes: 16 calories per 1/2 cup
- Serving People: 4 
'''

In [130]:
ASSISTANT_INSTRUCTION = f'''
You are helpful diabetic-friendly recipe assistant who give different recipe each time when user ask. Recipe can be for vegetarian, vegan, and non-vegeterian diets.You generate diabetic friendly recipe in below format:
<recipe>
    <recipe_name> {recipe_name} </recipe_name>
    <ingredients> {ingredients} </ingredients>
    <directions> {directions} </directions>
    <nutrition> {nutrients} </nutrition> 
    <total_calories_estimation>{total_calorie_estimation}</total_calories_estimation>
</recipe>
Always use above format to give recipe. Give total calories for recipe and refer USDA Food API for calories.
'''

In [131]:
print(ASSISTANT_INSTRUCTION)


You are helpful diabetic-friendly recipe assistant who give different recipe each time when user ask. Recipe can be for vegetarian, vegan, and non-vegeterian diets.You generate diabetic friendly recipe in below format:
<recipe>
    <recipe_name> Classic Apple Pie </recipe_name>
    <ingredients> 
- quinoa: 1 cup
- water: 2 cups
- black beans (rinsed and drained): 1 can (15 ounces)
- red bell pepper (diced): 1
- fresh cilantro (chopped): 1/4 cup
- lime juice: 1/4 cup
- olive oil: 2 tablespoons
- ground cumin: 1 teaspoon
- salt: 1/2 teaspoon
- black pepper: 1/4 teaspoon
- avocado (diced, optional): 1
- cherry tomatoes (halved, optional): 1/2 cup
 </ingredients>
    <directions> 
1. Cook Quinoa: Rinse the quinoa under cold water. In a medium saucepan, bring 2 cups of water to a boil. Add quinoa, reduce heat to low, cover, and simmer for about 15 minutes, or until quinoa is tender and water is absorbed. Remove from heat and let it cool.
2. Combine Ingredients: In a large bowl, combine the

In [132]:
ASSISTANT_NAME = 'Recipe Assistant'

In [133]:
from openai import OpenAI
import os

In [134]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [135]:
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", ""))


assistant = client.beta.assistants.create(
    name=ASSISTANT_NAME,
    instructions=ASSISTANT_INSTRUCTION,
    model="gpt-4-1106-preview",
)
show_json(assistant)

{'id': 'asst_D7EdOlnOfpsk6qH3FcaWpQj6',
 'created_at': 1712633791,
 'description': None,
 'file_ids': [],
 'instructions': '\nYou are helpful diabetic-friendly recipe assistant who give different recipe each time when user ask. Recipe can be for vegetarian, vegan, and non-vegeterian diets.You generate diabetic friendly recipe in below format:\n<recipe>\n    <recipe_name> Classic Apple Pie </recipe_name>\n    <ingredients> \n- quinoa: 1 cup\n- water: 2 cups\n- black beans (rinsed and drained): 1 can (15 ounces)\n- red bell pepper (diced): 1\n- fresh cilantro (chopped): 1/4 cup\n- lime juice: 1/4 cup\n- olive oil: 2 tablespoons\n- ground cumin: 1 teaspoon\n- salt: 1/2 teaspoon\n- black pepper: 1/4 teaspoon\n- avocado (diced, optional): 1\n- cherry tomatoes (halved, optional): 1/2 cup\n </ingredients>\n    <directions> \n1. Cook Quinoa: Rinse the quinoa under cold water. In a medium saucepan, bring 2 cups of water to a boil. Add quinoa, reduce heat to low, cover, and simmer for about 15 m

In [136]:
ASSISTANT_ID='asst_D7EdOlnOfpsk6qH3FcaWpQj6'

In [137]:
from bs4 import BeautifulSoup
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", ""))

def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )


def get_response(thread):
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    for m in messages:
        if m.role == 'assistant':
            return m.content[0].text.value
# Write recipe in xml format using Beautiful soup
def write_recipe(id: str, recipe_content:str):
    soup = BeautifulSoup(recipe_content, "xml")
    with open(f'../data/generated_recipe_v2/{id}.xml', 'w') as f:
        f.write(soup.prettify())

def create_thread_and_run(assistant_id: str, user_input:str):
    thread = client.beta.threads.create()
    run = submit_message(assistant_id, thread, user_input)
    return thread, run

In [138]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [142]:
prompts = [
    "Can you share a diabetic-friendly recipe for a hearty morning meal that's both satisfying and nutritious?",
    "I'm looking for a diabetic-friendly lunch option that's easy to prepare and delicious. Any suggestions?",
    "What's a diabetic-friendly recipe for dinner that's full of flavor and incorporates plenty of vegetables and lean proteins?",
    "Do you have a diabetic-friendly recipe for a quick and energizing breakfast that doesn't spike blood sugar levels?",
    "I'm in need of a diabetic-friendly lunch recipe that's suitable for meal prep and can be enjoyed on-the-go. Any recommendations?",
    "Could you recommend a diabetic-friendly dinner recipe that's perfect for entertaining guests, yet won't compromise on taste or healthiness?",
    "I'm searching for a diabetic-friendly breakfast recipe that's warm and comforting, ideal for those chilly mornings. Any ideas?",
    "What's a diabetic-friendly recipe for a light yet filling lunch option that's perfect for busy weekdays?",
    "Can you suggest a diabetic-friendly recipe for dinner that's family-friendly and can be customized to suit different taste preferences?",
    "Do you have a diabetic-friendly breakfast recipe that's suitable for someone with a sweet tooth but won't cause a sugar spike?",
    "Could you provide a recipe for a diabetic-friendly comfort dinner that's both warming and balanced?",
    "Can you suggest a diabetic-friendly snack that's perfect for on-the-go and packed with energy?",
    "Could you create a diabetic-friendly dessert recipe that satisfies sweet cravings without spiking blood sugar?",
    "Can you recommend a diabetic-friendly smoothie recipe that incorporates green vegetables for a nutrient boost?",
    "Could you devise a diabetic-friendly lunch recipe that's easy to pack for work and keeps well?",
    "Can you outline a diabetic-friendly appetizer that's ideal for entertaining and sure to impress guests?",
    "Could you suggest a diabetic-friendly soup recipe that's both comforting and full of flavor?",
    "Can you provide a recipe for a diabetic-friendly salad dressing that elevates any vegetable dish?",
    "Could you recommend a diabetic-friendly pasta alternative dish that's satisfying and low in carbohydrates?",
    "Can you share a diabetic-friendly recipe for a refreshing summer beverage that's hydrating and flavorful?"
]

In [144]:
len(prompts)

20

In [169]:
for i in range(19,20):
    thread, run = create_thread_and_run(assistant_id= ASSISTANT_ID, user_input=prompts[i-1])
    run = wait_on_run(thread=thread, run = run)
    recipe_content = get_response(thread=thread)
    write_recipe(id=str(i), recipe_content=recipe_content)
    recipe_id = str(i)
    print(f'Generated recipe for id:{recipe_id}')

Generated recipe for id:19


In [45]:
import re

# Function to extract nutritional values
def extract_nutrition_values(text):
    # Dictionary to hold the extracted values
    nutrition_values = {}

    # Regular expressions for extracting fat, carbohydrate, and protein
    patterns = {
        'fat': r"Total Fat (\d+\.?\d*g)",
        'carbohydrate': r"Total Carbohydrate (\d+\.?\d*g)",
        'protein': r"Protein (\d+\.?\d*g)",
    }

    # Loop through the patterns and search in the text
    for key, pattern in patterns.items():
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            # If a match is found, add it to the dictionary
            nutrition_values[key] = match.group(1)
        else:
            # If no match is found, set the value to None
            nutrition_values[key] = None

    return nutrition_values

In [147]:
import glob
from bs4 import  BeautifulSoup
file_paths = glob.glob(pathname='../data/generated_recipe_v2/*.xml')

In [162]:
file_paths

['../data/generated_recipe_v2/14.xml',
 '../data/generated_recipe_v2/15.xml',
 '../data/generated_recipe_v2/17.xml',
 '../data/generated_recipe_v2/16.xml',
 '../data/generated_recipe_v2/12.xml',
 '../data/generated_recipe_v2/13.xml',
 '../data/generated_recipe_v2/11.xml',
 '../data/generated_recipe_v2/20.xml',
 '../data/generated_recipe_v2/18.xml',
 '../data/generated_recipe_v2/19.xml']

In [163]:
with open(file_paths[0], 'r') as f:
    file = f.read() 

# 'xml' is the parser used. For html files, which BeautifulSoup is typically used for, it would be 'html.parser'.
soup = BeautifulSoup(file, 'xml')

In [164]:
soup.find('total_calories_estimation')

<total_calories_estimation>
  - Total Calories for entire recipe : Approximately 1194 calories for the entire recipe. 
- Calories for each ingredient:
    Chickpeas: 728 calories for 2 cups
    Cucumber: 16 calories for 1 medium
    Cherry Tomatoes: 32 calories per cup
    Red Onion: 15 calories per 1/4 cup
    Kalamata Olives: 45 calories per 1/4 cup
    Feta Cheese: 198 calories per 1/2 cup (if used)
    Extra-Virgin Olive Oil: 358 calories per 3 tablespoons
    Lemon Juice: 8 calories per 2 tablespoons
    Parsley: Negligible
    Dried Oregano, Salt, Black Pepper: Negligible
- Serving People: 4
 </total_calories_estimation>

In [149]:
print(soup.prettify())

<?xml version="1.0" encoding="utf-8"?>
<recipe>
 <recipe_name>
  Mediterranean Chickpea Salad
 </recipe_name>
 <ingredients>
  - chickpeas (cooked or canned): 2 cups
- cucumber (diced): 1 medium
- cherry tomatoes (halved): 1 cup
- red onion (finely chopped): 1/4 cup
- Kalamata olives (pitted and sliced): 1/4 cup
- feta cheese (crumbled, optional): 1/2 cup
- extra-virgin olive oil: 3 tablespoons
- lemon juice: 2 tablespoons
- fresh parsley (chopped): 1/4 cup
- dried oregano: 1 teaspoon
- salt: 1/4 teaspoon
- black pepper: 1/8 teaspoon
 </ingredients>
 <directions>
  1. Prepare Vegetables: Dice the cucumber, halve the cherry tomatoes, finely chop the red onion, slice the Kalamata olives, and chop the fresh parsley.
2. Combine Salad: In a large bowl, mix together chickpeas, diced cucumber, halved cherry tomatoes, chopped red onion, sliced Kalamata olives, and crumbled feta cheese if using.
3. Make Dressing: In a small bowl, whisk together extra-virgin olive oil, lemon juice, chopped fresh

In [155]:
from pydantic import BaseModel
from typing import  List, Optional
class Nutrition(BaseModel):
    fat: Optional[str] = None
    protein: Optional[str] = None
    carbohydrate: Optional[str] = None

class Recipe(BaseModel):
    id: str
    recipe_name:str
    ingredients: List[dict]
    directions: str
    nutrition: Nutrition
    total_calories_estimation: str 

class Recipes(BaseModel):
    recipes: List[Recipe] = []

In [156]:
def read_xml_file(file_path: str):
    with open(file_path, 'r') as f:
        file = f.read() 
        return file

def clean_text(content: str):
    clean_content = content.strip()
    return clean_content

def clean_ingredient(content: str):
    ingredients = []
    lines = content.split('\n')
    for line in lines:
        if len(line)>1:
            temp = {}
            if ':' in line:
                tokens = line.split(':')
                temp['name'] = tokens[0].replace('-','').strip()
                temp['quantity'] = tokens[1].strip()
            else:
                print(len(line))
                temp['name'] = line.replace('-','').strip()
                temp['quantity'] = 'NA'
            ingredients.append(temp)
    return ingredients

In [167]:
from pathlib import Path
def parse_llm_recipes(file_paths:list):
    recipes = Recipes()
    for file_path in file_paths:
        print(f'Parsing file:{file_path}')
        id = Path(file_path).stem
        file = read_xml_file(file_path)
        soup = BeautifulSoup(file, 'xml')
        recipe_name = soup.find('recipe_name')
        recipe_name = clean_text(recipe_name.text)
        ingredients = soup.find('ingredients')
        ingredients = clean_ingredient(ingredients.text)
        directions = soup.find('directions')
        directions = clean_text(directions.text)
        nutrition = soup.find('nutrition')
        nutrition = clean_text(nutrition.text)
        parsed_nutrition = extract_nutrition_values(text=nutrition)
        nutrition = Nutrition(fat=parsed_nutrition['fat'], protein=parsed_nutrition['protein'],carbohydrate=parsed_nutrition['carbohydrate'])
        total_calories_estimation = soup.find('total_calories_estimation')
        total_calories_estimation = clean_text(total_calories_estimation.text)
        recipe = Recipe(id = id, 
                        recipe_name = recipe_name, 
                        ingredients = ingredients,
                        directions=directions, 
                        nutrition=nutrition,
                        total_calories_estimation=total_calories_estimation)
        recipes.recipes.append(recipe) 
    return recipes 

In [170]:
llm_recipes = parse_llm_recipes(file_paths)

Parsing file:../data/generated_recipe_v2/14.xml
Parsing file:../data/generated_recipe_v2/15.xml
Parsing file:../data/generated_recipe_v2/17.xml
Parsing file:../data/generated_recipe_v2/16.xml
Parsing file:../data/generated_recipe_v2/12.xml
Parsing file:../data/generated_recipe_v2/13.xml
Parsing file:../data/generated_recipe_v2/11.xml
Parsing file:../data/generated_recipe_v2/20.xml
Parsing file:../data/generated_recipe_v2/18.xml
Parsing file:../data/generated_recipe_v2/19.xml


In [103]:
llm_recipes

Recipes(recipes=[Recipe(id='8', recipe_name='Spicy Grilled Chicken Salad', ingredients=[{'name': 'Chicken breasts', 'quantity': '2 (about 6 ounces each), boneless and skinless'}, {'name': 'Mixed salad greens', 'quantity': '4 cups'}, {'name': 'Cherry tomatoes', 'quantity': '1 cup, halved'}, {'name': 'Cucumber', 'quantity': '1, thinly sliced'}, {'name': 'Red onion', 'quantity': '1/4 cup, thinly sliced'}, {'name': 'Avocado', 'quantity': '1, sliced'}, {'name': 'Olive oil', 'quantity': '2 tbsp (plus 1 tsp for grilling)'}, {'name': 'Apple cider vinegar', 'quantity': '2 tbsp'}, {'name': 'Dijon mustard', 'quantity': '1 tsp'}, {'name': 'Garlic powder', 'quantity': '1/2 tsp'}, {'name': 'Paprika', 'quantity': '1/2 tsp'}, {'name': 'Dried oregano', 'quantity': '1/2 tsp'}, {'name': 'Salt', 'quantity': '1/4 tsp'}, {'name': 'Black pepper', 'quantity': '1/4 tsp'}, {'name': 'Lemon juice', 'quantity': 'from 1 lemon'}], directions='1. Preheat your grill to medium-high heat.\n2. In a small bowl, mix togeth

In [171]:
import json
with open('../data/generated_recipe_v2/gpt4_1106_recipes.json', 'w') as f:
    json.dump(llm_recipes.model_dump(mode='json'), f, indent=4, ensure_ascii=False)